In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
train = pd.read_csv('archive/sign_mnist_train.csv')
test = pd.read_csv('archive/sign_mnist_test.csv')

In [ ]:
train.head()

In [3]:
labels = train['label'].values

NameError: name 'train' is not defined

In [4]:
unique_val = np.array(labels)
np.unique(unique_val)

NameError: name 'labels' is not defined

In [5]:
plt.figure(figsize = (18,8))
sns.countplot(x =labels)

NameError: name 'sns' is not defined

<Figure size 1800x800 with 0 Axes>

In [ ]:
train.drop('label', axis = 1, inplace = True)

In [6]:
images = train.values
images = np.array([np.reshape(i, (28, 28)) for i in images])
images = np.array([i.flatten() for i in images])

NameError: name 'train' is not defined

In [7]:
from sklearn.preprocessing import LabelBinarizer

label_binrizer = LabelBinarizer()
labels = label_binrizer.fit_transform(labels)

NameError: name 'labels' is not defined

In [ ]:
labels

In [ ]:
index = 2
print(labels[index])
plt.imshow(images[index].reshape(28,28))

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.3, random_state = 101)

In [ ]:
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

batch_size = 128
num_classes = 24
epochs = 40

In [ ]:
X_train = x_train / 255
X_test = x_test / 255

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

plt.imshow(X_train[0].reshape(28,28))

In [ ]:
from keras import backend as K
from keras.optimizers import Adam

model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu', input_shape=(28, 28 ,1)))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.20))

model.add(Dense(num_classes, activation = 'softmax'))

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(), metrics = ['accuracy'])

In [ ]:
print(model.summary())

In [ ]:
history = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=epochs, batch_size=batch_size)

In [ ]:
model.save('hand_gesture_model.keras')
print('Model Saved')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')

plt.show()

In [ ]:
test_labels = test['label']
test.drop('label', axis = 1, inplace = True)

test_images = test.values
test_images = np.array([np.reshape(i, (28, 28)) for i in test_images])
test_images = np.array([i.flatten() for i in test_images])

test_labels = label_binrizer.fit_transform(test_labels)

test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

test_images.shape

y_pred = model.predict(test_images)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(test_labels, y_pred.round())

In [ ]:
def getLetter(result):
    classLabels = {0:'A', 1:'B', 2:'C', 3:'D', 4:'E', 5:'F', 6:'G', 7:'H', 8:'I', 9:'K', 10:'L', 11:'M', 12:'N', 13:'O', 14:'P', 15:'Q', 16:'R', 17:'S', 18:'T', 19:'U', 20:'V', 21:'W', 22:'X', 23:'Y'}
    try:
        res = int(result)
        return classLabels[res]
    except:
        return 'invalid'

In [ ]:
import cv2

cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print('Error: Unable to open webcam')
    exit()

while True:
    ret, frame = cap.read()


    frame = cv2.flip(frame, 1)

    roi = frame[100:400, 320:620]
    cv2.imshow('roi', roi)
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)

    cv2.imshow('roi scaled and gray', roi)
    copy = frame.copy()
    cv2.rectangle(copy, (320, 100), (620, 400), (255, 0, 0), 5)

    roi = roi.reshape(1, 28, 28, 1)

    result = np.argmax(model.predict(roi, 1, verbose=0), axis=1)
    

    cv2.putText(copy, getLetter(result), (300, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
    cv2.imshow('frame', copy)

    if cv2.waitKey(1) == 13:
        break

cap.release()
cv2.destroyAllWindows()
